In [1]:
import pandas as pd

import tensorflow_datasets as tfds

from tqdm import tqdm



dataset_train = tfds.load("imdb_reviews", split="train",shuffle_files=True)
dataset_test = tfds.load("imdb_reviews", split="test",shuffle_files=True)


2023-04-20 11:37:02.508561: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-04-20 11:37:02.509859: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-04-20 11:37:02.541252: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-04-20 11:37:02.541758: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-20 11:37:02.991474: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Co

In [2]:
def convert_to_df(dataset):
    data = [{"text":item["text"].numpy().decode("utf-8"), "label":item["label"].numpy()} for item in tqdm(dataset)]
    return pd.DataFrame(data)

In [3]:
df_train = convert_to_df(dataset_train)
df_test = convert_to_df(dataset_test)

  0%|                                                                                                                                              | 0/25000 [00:00<?, ?it/s]2023-04-20 11:37:03.516995: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype string and shape [1]
	 [[{{node Placeholder/_1}}]]
2023-04-20 11:37:03.517477: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_4' with dtype int64 and shape [1]
	 [[{{node Placeholder/_4}}]]
  0%|                                                                                                                                             

In [16]:
df_train["sentiment"] = [None]*len(df_train)
df_train["sentiment"] = df_train["label"].apply(sentiment) 

df_test["sentiment"] = [None]*len(df_test)
df_test["sentiment"] = df_test["label"].apply(sentiment) 


In [4]:
def sentiment(value):
    if value == 1:
        return "positive"
    return "negative"

In [5]:
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
nltk.download("stopwords")
STOPWORDS = stopwords.words("english")

[nltk_data] Downloading package stopwords to /home/molesz/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [6]:
from bs4 import BeautifulSoup


def remove_html_tags(text):
    s = BeautifulSoup(text)
    return s.get_text()

In [7]:
df_train["text"] = df_train["text"].apply(remove_html_tags)


In [8]:
nltk.download("punkt")

[nltk_data] Downloading package punkt to /home/molesz/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [9]:
tokenized_reviews =df_train["text"].apply(lambda review_text: word_tokenize(review_text.replace("\n","").lower() ))

In [10]:
tokenized_reviews.head()

0    [this, was, an, absolutely, terrible, movie, ....
1    [i, have, been, known, to, fall, asleep, durin...
2    [mann, photographs, the, alberta, rocky, mount...
3    [this, is, the, kind, of, film, for, a, snowy,...
4    [as, others, have, mentioned, ,, all, the, wom...
Name: text, dtype: object

In [11]:
d = dict()
for review in tqdm(tokenized_reviews):
    for word in review:
        if word not in STOPWORDS  and word.isalpha():
            d[word] = d.get(word,0) + 1

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 25000/25000 [00:05<00:00, 4222.92it/s]


In [12]:
d = sorted(d.items(), key=lambda item: item[1], reverse=True)
d[:10]

[('movie', 42586),
 ('film', 38763),
 ('one', 25574),
 ('like', 19664),
 ('good', 14612),
 ('would', 13378),
 ('even', 12452),
 ('time', 11904),
 ('really', 11682),
 ('story', 11559)]

In [13]:
DESIRED_VOCAB_SIZE = 4000

VOCAB = [k for k,v in d[:DESIRED_VOCAB_SIZE]]
word_table = pd.DataFrame({"word": VOCAB})
word_table.head(10)

,word
0,movie
1,film
2,one
3,like
4,good
5,would
6,even
7,time
8,really
9,story


In [18]:
dic_freq = {"positive": {}, "negative": {}}

VOCAB_IDX = {}
for i in range(0, len(word_table["word"].values)):
    VOCAB_IDX[word_table["word"].values[i]] = i
    
for idx in range(df_train.shape[0]):
    review = df_train.iloc[idx]["text"]
    sentiment = df_train.iloc[idx]["sentiment"]
    
    
    for word in review.split(" "):
        if word in VOCAB_IDX:
            dic_freq[sentiment][word] = dic_freq[sentiment].get(word,0) + 1

In [20]:
dic_freq

{'positive': {'kind': 1098,
  'film': 14412,
  'afternoon': 51,
  'rest': 698,
  'world': 1334,
  'go': 2009,
  'ahead': 148,
  'business': 225,
  'big': 1363,
  'couple': 692,
  'performances': 859,
  'row': 34,
  'plot': 1745,
  'dangerous': 129,
  'warm': 118,
  'witty': 114,
  'life': 2339,
  'family': 1359,
  'every': 1727,
  'sense': 824,
  'one': 10684,
  'deserves': 288,
  'praise': 68,
  'others': 566,
  'women': 586,
  'nude': 72,
  'mostly': 387,
  'absolutely': 601,
  'shows': 1254,
  'female': 358,
  'men': 683,
  'around': 1382,
  'want': 1472,
  'become': 881,
  'year': 724,
  'old': 1723,
  'boy': 518,
  'becomes': 718,
  'man': 1996,
  'really': 5158,
  'fast': 246,
  'lucky': 103,
  'gets': 1451,
  'courage': 88,
  'true': 1131,
  'seen': 2815,
  'anybody': 100,
  'interested': 319,
  'suffering': 96,
  'eating': 84,
  'amazingly': 78,
  'accurate': 125,
  'sensitive': 110,
  'portrayal': 336,
  'teenage': 154,
  'causes': 70,
  'girl': 902,
  'played': 1333,
  'brill